# LLM Client Testing Notebook

이 노트북은 LLM 클라이언트의 기능을 테스트하고 다양한 사용 사례를 시연합니다.

## Setup
먼저 필요한 모듈을 import합니다.

In [ ]:
import json
import sys
from pathlib import Path

from dotenv import load_dotenv

from src.app.llm import LLMClient

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Load environment variables
load_dotenv(project_root / ".env")

print("✓ Setup complete")

## 1. Basic Chat Completion

기본적인 채팅 완성 기능을 테스트합니다.

In [ ]:
# OpenAI 클라이언트 생성
openai_client = LLMClient(provider="openai", temperature=0.7)

messages = [
    {"role": "system", "content": "You are a helpful AI research assistant."},
    {"role": "user", "content": "What are the key trends in AI research in 2024?"},
]

response = openai_client.chat_completion(messages, max_tokens=300)
print("OpenAI Response:")
print(response)

## 2. Claude Chat Completion

Claude API를 사용한 채팅 완성을 테스트합니다.

In [ ]:
# Claude 클라이언트 생성 (API 키가 설정된 경우에만)
try:
    claude_client = LLMClient(provider="claude", temperature=0.7)

    messages = [{"role": "user", "content": "What are the key trends in AI research in 2024?"}]

    response = claude_client.chat_completion(messages, max_tokens=300)
    print("Claude Response:")
    print(response)
except Exception as e:
    print(f"Claude API not available: {e}")

## 3. JSON Response Format

JSON 형식의 응답을 요청하고 파싱합니다.

In [ ]:
client = LLMClient(provider="openai")

messages = [
    {"role": "system", "content": "You are a helpful assistant that responds in JSON format."},
    {
        "role": "user",
        "content": """
        Analyze this article title and return a JSON object with:
        - category: one of (paper, news, report)
        - importance_score: 0.0 to 1.0
        - keywords: list of 3-5 relevant keywords
        - summary: brief one-line summary

        Title: "GPT-5 Achieves Human-Level Performance on Complex Reasoning Tasks"
        """,
    },
]

response = client.chat_completion(messages, response_format="json", max_tokens=300)
print("JSON Response:")
print(response)

# Parse JSON
data = json.loads(response)
print("\nParsed Data:")
print(json.dumps(data, indent=2))

## 4. Article Summarization (Korean)

논문을 한국어로 요약하는 기능을 테스트합니다.

In [ ]:
client = LLMClient(provider="openai", temperature=0.5)

article_text = """
Researchers at MIT have developed a new neural network architecture
that combines the benefits of transformers with the efficiency of
convolutional neural networks. The hybrid model, dubbed TransConv,
achieves state-of-the-art results on image classification tasks while
requiring 40% less computational resources than traditional transformer
models. The key innovation lies in the selective attention mechanism
that adaptively chooses between local and global feature processing
based on the input characteristics.
"""

messages = [
    {"role": "system", "content": "You are an AI research expert. Summarize articles in Korean."},
    {
        "role": "user",
        "content": f"다음 논문 내용을 한국어로 3-4문장으로 요약해주세요:\n\n{article_text}",
    },
]

summary = client.chat_completion(messages, max_tokens=500)
print("Korean Summary:")
print(summary)

## 5. Embedding Generation

텍스트 임베딩을 생성하고 유사도를 계산합니다.

In [ ]:
import numpy as np

client = LLMClient(provider="openai")

# 여러 텍스트의 임베딩 생성
texts = [
    "Transformer architecture in deep learning",
    "Attention mechanism for neural networks",
    "Reinforcement learning for robotics",
    "Computer vision using CNNs",
]

embeddings = [client.generate_embedding(text) for text in texts]

print(f"Generated {len(embeddings)} embeddings")
print(f"Embedding dimension: {len(embeddings[0])}")
print(f"First embedding (first 5 values): {embeddings[0][:5]}")

In [ ]:
# 코사인 유사도 계산
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


print("\nCosine Similarities:")
for i, text_i in enumerate(texts):
    for j, text_j in enumerate(texts):
        if i < j:
            similarity = cosine_similarity(embeddings[i], embeddings[j])
            print(f"'{text_i}' vs '{text_j}': {similarity:.4f}")

## 6. Async Operations

비동기 작업을 테스트합니다.

In [ ]:
import asyncio


async def test_async_operations():
    client = LLMClient(provider="openai")

    # Async chat completion
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is machine learning in one sentence?"},
    ]

    response = await client.achat_completion(messages, max_tokens=100)
    print("Async Chat Response:")
    print(response)
    print()

    # Async embedding
    embedding = await client.agenerate_embedding("Deep learning for NLP")
    print(f"Async Embedding dimension: {len(embedding)}")
    print(f"First 5 values: {embedding[:5]}")


# Run async function
await test_async_operations()

## 7. Batch Processing

여러 문서를 동시에 처리합니다.

In [ ]:
async def process_articles_batch(articles):
    """여러 논문을 동시에 요약"""
    client = LLMClient(provider="openai", temperature=0.5)

    async def summarize_one(article):
        messages = [
            {"role": "system", "content": "Summarize the article in Korean in 2 sentences."},
            {"role": "user", "content": article},
        ]
        return await client.achat_completion(messages, max_tokens=200)

    # 모든 요약을 동시에 실행
    summaries = await asyncio.gather(*[summarize_one(article) for article in articles])
    return summaries


# Test batch processing
articles = [
    "GPT-5 shows remarkable improvements in reasoning capabilities.",
    "New transformer architecture reduces computational costs by 40%.",
    "Reinforcement learning breakthrough enables better robot control.",
]

summaries = await process_articles_batch(articles)

print("Batch Processing Results:")
for i, (article, summary) in enumerate(zip(articles, summaries, strict=False), 1):
    print(f"\n{i}. Original: {article}")
    print(f"   Summary: {summary}")

## 8. Temperature Comparison

다양한 temperature 값에 따른 응답 차이를 비교합니다.

In [ ]:
temperatures = [0.0, 0.5, 1.0]
prompt = "Explain neural networks in one sentence."

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

print("Temperature Comparison:")
print(f"Prompt: {prompt}\n")

for temp in temperatures:
    client = LLMClient(provider="openai", temperature=temp)
    response = client.chat_completion(messages, max_tokens=100)
    print(f"Temperature {temp}:")
    print(f"  {response}")
    print()

## 9. Error Handling

에러 처리를 테스트합니다.

In [ ]:
# Test invalid provider
try:
    invalid_client = LLMClient(provider="invalid")
except ValueError as e:
    print(f"✓ Caught expected error for invalid provider: {e}")

# Test with very large max_tokens (should be handled by API)
try:
    client = LLMClient(provider="openai")
    messages = [{"role": "user", "content": "Hello"}]
    response = client.chat_completion(messages, max_tokens=100000)
    print("\nResponse with large max_tokens received (API handled it)")
except Exception as e:
    print(f"Error with large max_tokens: {e}")

## 10. Use Case: Research Article Analysis

실제 사용 사례: 연구 논문 분석 파이프라인

In [ ]:
async def analyze_research_article(title, abstract):
    """연구 논문을 분석하고 메타데이터를 추출"""
    client = LLMClient(provider="openai", temperature=0.3)

    # 1. 카테고리 분류 및 중요도 평가
    analysis_messages = [
        {
            "role": "system",
            "content": "You are an AI research expert. Analyze research articles and respond in JSON.",
        },
        {
            "role": "user",
            "content": f"""
            Analyze this research article and return JSON with:
            - category: one of (paper, news, report)
            - importance_score: 0.0 to 1.0
            - keywords: list of 5 keywords
            - field: research field (e.g., "Computer Vision", "NLP", "Robotics")

            Title: {title}
            Abstract: {abstract}
            """,
        },
    ]

    # 2. 한국어 요약
    summary_content = (
        f"다음 논문을 한국어로 3-4문장으로 요약해주세요:\n\n" f"Title: {title}\n\nAbstract: {abstract}"
    )
    summary_messages = [
        {"role": "system", "content": "You are an AI research expert. Summarize in Korean."},
        {
            "role": "user",
            "content": summary_content,
        },
    ]

    # 3. 임베딩 생성
    embedding_text = f"{title} {abstract}"

    # 모두 비동기로 동시 실행
    analysis_task = client.achat_completion(analysis_messages, response_format="json", max_tokens=300)
    summary_task = client.achat_completion(summary_messages, max_tokens=500)
    embedding_task = client.agenerate_embedding(embedding_text)

    analysis_response, summary, embedding = await asyncio.gather(
        analysis_task, summary_task, embedding_task
    )

    # 결과 파싱
    analysis = json.loads(analysis_response)

    return {
        "title": title,
        "category": analysis.get("category"),
        "importance_score": analysis.get("importance_score"),
        "keywords": analysis.get("keywords", []),
        "field": analysis.get("field"),
        "summary_korean": summary,
        "embedding_dim": len(embedding),
        "embedding_preview": embedding[:5],
    }


# Test the analysis pipeline
test_article = {
    "title": "Attention Is All You Need",
    "abstract": """
    The dominant sequence transduction models are based on complex recurrent or
    convolutional neural networks in an encoder-decoder configuration. The best
    performing models also connect the encoder and decoder through an attention
    mechanism. We propose a new simple network architecture, the Transformer,
    based solely on attention mechanisms, dispensing with recurrence and convolutions
    entirely.
    """,
}

result = await analyze_research_article(test_article["title"], test_article["abstract"])

print("Research Article Analysis Result:")
print(json.dumps(result, indent=2, ensure_ascii=False))

## Summary

이 노트북에서 다룬 내용:

1. ✓ Basic chat completion (OpenAI & Claude)
2. ✓ JSON response format
3. ✓ Korean summarization
4. ✓ Embedding generation and similarity
5. ✓ Async operations
6. ✓ Batch processing
7. ✓ Temperature comparison
8. ✓ Error handling
9. ✓ Complete research article analysis pipeline

LLM 클라이언트가 정상적으로 작동하며, 다양한 사용 사례에 활용할 수 있습니다.